# Crime since 2001 in Chicago

* Data source: [data.gov](https://catalog.data.gov/dataset/crimes-2001-to-present-398a4)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime

In [6]:
%%time

df = spark.read.format('csv')\
    .option('header', 'true')\
    .load('hdfs://orion11:13030/crime-since-2001-chicago.csv')\
    .limit(100)
df.show(1)

+--------+-----------+--------------------+---------------+----+------------+-------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|          Block|IUCR|Primary Type|        Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+---------------+----+------------+-------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10000092|   HY189866|03/18/2015 07:44:...|047XX W OHIO ST|041A|     BATTERY|AGGRAVATED: HANDGUN|              STREET| false|   false|1111|     011|  28|            2